In [30]:
import pandas as pd
import numpy as np
import re

In [3]:
data = pd.read_csv('thanksgiving.csv', encoding= 'Latin-1')

In [4]:
data.head(3)

,RespondentID,Do you celebrate Thanksgiving?,What is typically the main dish at your Thanksgiving dinner?,What is typically the main dish at your Thanksgiving dinner? - Other (please specify),How is the main dish typically cooked?,How is the main dish typically cooked? - Other (please specify),What kind of stuffing/dressing do you typically have?,What kind of stuffing/dressing do you typically have? - Other (please specify),What type of cranberry saucedo you typically have?,What type of cranberry saucedo you typically have? - Other (please specify),...,Have you ever tried to meet up with hometown friends on Thanksgiving night?,"Have you ever attended a ""Friendsgiving?""",Will you shop any Black Friday sales on Thanksgiving Day?,Do you work in retail?,Will you employer make you work on Black Friday?,How would you describe where you live?,Age,What is your gender?,How much total combined money did all members of your HOUSEHOLD earn last year?,US Region
0,4337954960,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,None,NaN,...,Yes,No,No,No,NaN,Suburban,18 - 29,Male,"$75,000 to $99,999",Middle Atlantic
1,4337951949,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,Other (please specify),Homemade cranberry gelatin ring,...,No,No,Yes,No,NaN,Rural,18 - 29,Female,"$50,000 to $74,999",East South Central
2,4337935621,Yes,Turkey,NaN,Roasted,NaN,Rice-based,NaN,Homemade,NaN,...,Yes,Yes,Yes,No,NaN,Suburban,18 - 29,Male,"$0 to $9,999",Mountain


In [17]:
data.iloc[:,1].value_counts()

Yes    980
No      78
Name: Do you celebrate Thanksgiving?, dtype: int64

In [20]:
data = data.loc[data.iloc[:,1] == 'Yes',:]

In [21]:
len(data)

980

In [25]:
data.loc[:,'What is typically the main dish at your Thanksgiving dinner?'].value_counts()

Turkey                    859
Other (please specify)     35
Ham/Pork                   29
Tofurkey                   20
Chicken                    12
Roast beef                 11
I don't know                5
Turducken                   3
Name: What is typically the main dish at your Thanksgiving dinner?, dtype: int64

In [27]:
data.loc[data.iloc[:,2] == 'Tofurkey','Do you typically have gravy?'].value_counts()

Yes    12
No      8
Name: Do you typically have gravy?, dtype: int64

In [28]:
data.loc[:, 'How would you describe where you live?'].value_counts()

Suburban    496
Urban       236
Rural       216
Name: How would you describe where you live?, dtype: int64

In [44]:
ate_pies = data.filter(regex='Which type of pie is typically served at your Thanksgiving dinner\\? Please select all that apply\\. - (Apple|Pumpkin|Pecan)').isnull()

In [51]:
ate_pies = ate_pies.set_index(data.RespondentID)

In [65]:
print('APPLE PIES\n\n', ate_pies.iloc[:,0].value_counts())

APPLE PIES

 False    514
True     466
Name: Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Apple, dtype: int64


In [66]:
print('PECAN PIES\n\n', ate_pies.iloc[:,1].value_counts())

PECAN PIES

 True     638
False    342
Name: Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pecan, dtype: int64


In [67]:
print('PUMPKIN PIES\n\n', ate_pies.iloc[:,2].value_counts())

PUMPKIN PIES

 False    729
True     251
Name: Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pumpkin, dtype: int64


In [73]:
need_pies = ate_pies.apply(sum, axis=1)
print('Don\'t eat pies ', len(need_pies[need_pies == 0]))

Don't eat pies  137


In [102]:
def prepare_age(string):
    return int(re.sub(pattern='\+', repl='', string=string).split(' ')[0])

In [103]:
data['int_age'] = data.Age[~data.Age.isnull()].apply(prepare_age)    

In [104]:
data['int_age'].value_counts()

45.0    269
60.0    258
30.0    235
18.0    185
Name: int_age, dtype: int64

In [114]:
data['int_age'].describe()

count    947.000000
mean      40.089757
std       15.352014
min       18.000000
25%       30.000000
50%       45.000000
75%       60.000000
max       60.000000
Name: int_age, dtype: float64

In [111]:
def prepare_salary(string):
    if re.search('Prefer', string):
        return None
    else:
        return int(re.sub(pattern='\$|,', repl='', string=string).split(' ')[0])

In [112]:
data['int_income'] = data.loc[~data.loc[:, 'How much total combined money did all members of your HOUSEHOLD earn last year?'].isnull(),
                              'How much total combined money did all members of your HOUSEHOLD earn last year?'].apply(prepare_salary) 

In [115]:
data['int_income'].value_counts()

25000.0     166
75000.0     127
50000.0     127
100000.0    109
200000.0     76
10000.0      60
0.0          52
125000.0     48
150000.0     38
175000.0     26
Name: int_income, dtype: int64

In [113]:
data['int_income'].describe()

count       829.000000
mean      75965.018094
std       59068.636748
min           0.000000
25%       25000.000000
50%       75000.000000
75%      100000.000000
max      200000.000000
Name: int_income, dtype: float64

In [116]:
data.loc[data['int_income'] < 150000,
         'How far will you travel for Thanksgiving?'].value_counts()

Thanksgiving is happening at my home--I won't travel at all                         281
Thanksgiving is local--it will take place in the town I live in                     203
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    150
Thanksgiving is out of town and far away--I have to drive several hours or fly       55
Name: How far will you travel for Thanksgiving?, dtype: int64

In [118]:
data.loc[data['int_income'] >= 150000,
         'How far will you travel for Thanksgiving?'].value_counts()

Thanksgiving is happening at my home--I won't travel at all                         66
Thanksgiving is local--it will take place in the town I live in                     34
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    25
Thanksgiving is out of town and far away--I have to drive several hours or fly      15
Name: How far will you travel for Thanksgiving?, dtype: int64

In [121]:
data.pivot_table(index = 'Have you ever tried to meet up with hometown friends on Thanksgiving night?',
                 columns = 'Have you ever attended a "Friendsgiving?"',
                 values = 'int_age')

"Have you ever attended a ""Friendsgiving?""",No,Yes
Have you ever tried to meet up with hometown friends on Thanksgiving night?,,
No,42.283702,37.010526
Yes,41.475410,33.976744


In [ ]:
data.pivot_table(index = 'Have you ever tried to meet up with hometown friends on Thanksgiving night?',
                 columns = 'Have you ever attended a "Friendsgiving?"',
                 values = 'int_age')